In [1]:
import numpy.random as rand
import numpy as np

In [2]:
def theta_policy(n_states, n_actions) : 
    theta = np.zeros((n_states, n_actions))    
    return theta

$$ \theta  - \textrm{ parametrized policy}, \theta \in R^{ n \times k }, \textrm{where} $$
$$ n=|S| - \text{number of states} ,$$
$$ k=|A| - \text{number of actions} , $$
$$ \theta_{ij}   - \textrm{ policy for state i  and action j }$$


In [3]:
theta = theta_policy(5, 2)
print theta

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [4]:
def mu_policy( theta  ):
    n_states, n_actions = theta.shape
    mu  = np.zeros((n_states, n_actions))
    for state in range(0, n_states) : 
        max_theta = np.max(theta[state])
        mu[state] = np.exp(theta[state]-max_theta) / np.sum(np.exp(theta[state] - max_theta))
    return mu

$$ \mu - \text{softmax policy} $$
$$ $$ 
$$ \mu(a=j | s=i; \theta) = \mu_{\theta}(i,j) $$
$$ $$
$$  \mu_{\theta}(i,j)   =  \frac{\exp{\theta_{ij}}}{\sum_t{\exp{\theta_{it}}}} \ $$

$\text{ computed using sum-exp trick} : $

$$  \frac{\exp{\theta_{ij}}}{\sum_t{\exp{\theta_{it}}}} = \frac{\exp{\theta_{ij}} \times \exp{(-a)}}{\sum_t{\exp{\theta_{it}}} \times \exp{(-a)} } =  \frac{\exp{(\theta_{ij} - a)}}{\sum_t{\exp{(\theta_{it} - a)}}} $$

$$ a = \max_{t}{\theta_{it}} $$

In [5]:
mu = mu_policy(theta)
print mu

[[ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]]


In [6]:
def log_mu_policy( theta  ):
    n_states, n_actions = theta.shape
    log_mu  = np.zeros((n_states, n_actions))
    for state in range(0, n_states) : 
        max_theta = np.max(theta[state])
        log_sum_exp = max_theta  +  np.log(np.sum(np.exp(theta[state] - max_theta)))
        log_mu[state] = theta[state] - log_sum_exp
    return log_mu

$$ \log \mu_{\theta}(i,j)   = \log \frac{\exp{\theta_{ij}}}{\sum_t{\exp{\theta_{it}}}} \  = \theta_{ij}  - \log \sum_t{\exp{\theta_{it}}}  = \theta_{ij} - ( a + \log \sum_t{\exp{(\theta_{it}-a})}  )$$

In [7]:
log_mu = log_mu_policy(theta)
print log_mu

[[-0.69314718 -0.69314718]
 [-0.69314718 -0.69314718]
 [-0.69314718 -0.69314718]
 [-0.69314718 -0.69314718]
 [-0.69314718 -0.69314718]]


In [8]:
def gradient_log_mu_policy(theta, mu ,  state, action) :
    n_states, n_actions = theta.shape
    grad  = np.zeros((n_states, n_actions))
    grad[state, action] = 1 
    max_theta = np.max(theta[state])
    grad[state] = grad[state] - mu[state]
    return grad

$$ \nabla \log \mu_{\theta}(i,j)  = \nabla( \theta_{ij}  - \log \sum_t{\exp{\theta_{it}}} )  = \begin{pmatrix}
  \frac{\partial \log \mu_{\theta}(i,j)} {\partial \theta_{11}}  &  \frac{\partial \log \mu_{\theta}(i,j)} {\partial \theta_{12}} & \cdots &  \frac{\partial \log \mu_{\theta}(i,j)} {\partial \theta_{1m}} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  \frac{\partial \log \mu_{\theta}(i,j)} {\partial \theta_{n1}} &  \frac{\partial \log \mu_{\theta}(i,j)} {\partial \theta_{n2}} & \cdots & \frac{\partial \log \mu_{\theta}(i,j)} {\partial \theta_{nm}} 
 \end{pmatrix} $$

$$ \frac{\partial ( \log \sum_t{\exp{\theta_{pt}}} ) } {\partial \theta_{pl}} =  \frac{\exp{\theta_{pl}}}{\sum_t{\exp{\theta_{pt}}} }  = \mu_{\theta}(p,l) $$

$$ \frac{\partial \log \mu_{\theta}(i,j)} {\partial \theta_{pl}} =  \frac{\partial ( \theta_{ij}  - \log \sum_t{\exp{\theta_{it}}} ) } {\partial \theta_{pl}}  =  \begin{cases}
    1 -  \mu_{\theta}(p,l)      & \quad \text{if } i=p, j=l \\
    -  \mu_{\theta}(p,l)     & \quad \text{if } i=p, j \ne l \\
    0                           & \quad \text{if }  i \ne p, j \ne l  \\
  \end{cases} $$

In [9]:
print "gradient log mu :\n", gradient_log_mu_policy(theta, mu, 1, 0)
print "mu policy after adding gradient :\n" , mu_policy(theta + gradient_log_mu_policy(theta, mu, 1, 1))

gradient log mu :
[[ 0.   0. ]
 [ 0.5 -0.5]
 [ 0.   0. ]
 [ 0.   0. ]
 [ 0.   0. ]]
mu policy after adding gradient :
[[ 0.5         0.5       ]
 [ 0.26894142  0.73105858]
 [ 0.5         0.5       ]
 [ 0.5         0.5       ]
 [ 0.5         0.5       ]]


In [10]:
def path_likelihood_ratio(theta_policy, mu_policy, path_states,  path_actions ) :
    likelihood_ratio = np.zeros(theta_policy.shape)
    path_len = len(path_states)
    for t in range(0, path_len) :
        likelihood_ratio = likelihood_ratio + gradient_log_mu_policy(theta_policy, mu_policy , path_states[t] , path_actions[t])
    return likelihood_ratio

Likelihood ratio of path: 
$$ u(\xi;\theta)  =  \sum_t{ \nabla \log \mu_{\theta}(s_t,a_t) } $$

In [18]:
print mu
for i in range(0, paths_states.shape[0]) :
    print "path state    : ",paths_states[i]
    print "path  actions : ",paths_actions[i]
    print "path reward : ",paths_rewards[i]
    print "path likelihood ratio: \n", path_likelihood_ratio(theta, mu, paths_states[i], paths_actions[i])

[[ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]]
path state    :  [2 0 1 2 3]
path  actions :  [1 0 0 0 1]
path reward :  1.6561
path likelihood ratio: 
[[ 0.5 -0.5]
 [ 0.5 -0.5]
 [ 0.   0. ]
 [-0.5  0.5]
 [ 0.   0. ]]
path state    :  [3 0 1 0 0]
path  actions :  [1 0 1 0 1]
path reward :  1.81
path likelihood ratio: 
[[ 0.5 -0.5]
 [-0.5  0.5]
 [ 0.   0. ]
 [-0.5  0.5]
 [ 0.   0. ]]
path state    :  [2 3 4 4 4]
path  actions :  [0 0 0 0 1]
path reward :  7.4682
path likelihood ratio: 
[[ 0.   0. ]
 [ 0.   0. ]
 [ 0.5 -0.5]
 [ 0.5 -0.5]
 [ 0.5 -0.5]]
path state    :  [2 0 1 2 3]
path  actions :  [0 0 0 0 0]
path reward :  0.0
path likelihood ratio: 
[[ 0.5 -0.5]
 [ 0.5 -0.5]
 [ 1.  -1. ]
 [ 0.5 -0.5]
 [ 0.   0. ]]
path state    :  [0 0 1 2 0]
path  actions :  [1 0 0 1 0]
path reward :  0.729
path likelihood ratio: 
[[ 0.5 -0.5]
 [ 0.5 -0.5]
 [-0.5  0.5]
 [ 0.   0. ]
 [ 0.   0. ]]
path state    :  [2 0 1 0 1]
path  actions :  [1 0 1 0 0]
path reward :  1.81
path likelih

In [19]:
def policy_gradient(theta_policy, mu_policy , paths_states , paths_actions, paths_rewards) :
    policy_grad = np.zeros(theta_policy.shape)
    n_paths , path_len =  paths_states.shape
    for path in range(0, n_paths) :
        policy_grad = policy_grad + paths_rewards[path]*path_likelihood_ratio(theta_policy, mu_policy, paths_states[path], paths_actions[path])
    policy_grad = policy_grad/n_paths
    return policy_grad

Policy gradient estimation :  
$$ \nabla \eta(\theta)  =  \frac{1}{M} \sum_m \rho(\xi_m) \sum_t{ \nabla \log \mu_{\theta}(s_{mt},a_{mt}) } $$

In [20]:
print "current policy mu : \n", mu_policy(theta)
print "generated trajectories : \n states :\n" , paths_states, '\n actions :\n',  paths_actions, '\n rewards :\n', paths_rewards
print "policy gradient : \n",  policy_gradient(theta , mu, paths_states, paths_actions , paths_rewards)
print "new mu policy : \n",  mu_policy(theta + policy_gradient(theta , mu, paths_states, paths_actions , paths_rewards))

current policy mu : 
[[ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]]
generated trajectories : 
 states :
[[2 0 1 2 3]
 [3 0 1 0 0]
 [2 3 4 4 4]
 [2 0 1 2 3]
 [0 0 1 2 0]
 [2 0 1 0 1]
 [4 0 1 0 1]
 [3 0 0 1 0]
 [1 2 3 0 1]
 [1 2 0 0 0]] 
 actions :
[[1 0 0 0 1]
 [1 0 1 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [1 0 0 1 0]
 [1 0 1 0 0]
 [1 0 1 0 1]
 [1 1 0 1 0]
 [0 0 1 0 0]
 [0 1 1 1 1]] 
 rewards :
[ 1.6561  1.81    7.4682  0.      0.729   1.81    3.4661  1.729   0.81    0.9   ]
policy gradient : 
[[ 0.729315 -0.729315]
 [-0.278305  0.278305]
 [ 0.24196  -0.24196 ]
 [ 0.073155 -0.073155]
 [ 0.200105 -0.200105]]
new mu policy : 
[[ 0.81132305  0.18867695]
 [ 0.3643322   0.6356678 ]
 [ 0.6186731   0.3813269 ]
 [ 0.53651239  0.46348761]
 [ 0.59873811  0.40126189]]


In [21]:
def policy_gradient_algo(transitions, rewards, discount, path_len=10,  n_paths=100, gamma=1.0, eps=0.01, n_iterations=100, logging=False) : 
    n_states, n_actions = rewards.shape
    theta = theta_policy(n_states, n_actions)
    mu = mu_policy(theta)
    n=0
    paths_states, paths_actions, paths_rewards = generate_rollouts(mu, transitions, rewards, discount , path_len, n_paths )
    pgrad = policy_gradient(theta, mu,  paths_states, paths_actions, paths_rewards) 
    theta_diff =  (gamma/(n+1))*pgrad 
    theta_diff_norm = np.linalg.norm(theta_diff)
    #mu_diff = np.linalg.norm(mu_policy(theta) - mu_policy(theta+pgrad))
    while ( (n<n_iterations) & (theta_diff_norm>eps) ):
        if (logging) :
            print "mu policy : \n",  mu_policy(theta)
            print "policy gradient: \n", pgrad
            print "theta policy : \n" , theta_diff
            print "theta policy diff norm: " , theta_diff_norm
        theta_diff =  (gamma/(n+1))*pgrad
        theta = theta + theta_diff
        mu = mu_policy(theta)
        paths_states, paths_actions, paths_rewards = generate_rollouts(mu_policy(theta), transitions, rewards, discount , path_len, n_paths )
        pgrad = policy_gradient(theta, mu , paths_states, paths_actions, paths_rewards)
        #mu_diff = np.linalg.norm(mu_policy(theta) - mu_policy(theta+pgrad))
        theta_diff_norm = np.linalg.norm(theta_diff) 
        n = n+1
    return theta

repeat
$$ \theta^{n+1} = \theta^{n} + \frac {\gamma}{n}  * \nabla \eta(\theta^n) $$
until 
$$ || \theta_{n+1} - \theta_{n} || > \epsilon$$

In [22]:
policy_gradient_algo(P, R , discount,  path_len=10, gamma = 10.0)

array([[ 5.34621791, -5.34621791],
       [-5.74054404,  5.74054404],
       [ 2.59020224, -2.59020224],
       [ 2.99125497, -2.99125497],
       [ 4.46383365, -4.46383365]])

Now let's compare with other existing method  - policy iteration

In [27]:
import mdptoolbox_copy
pi =  mdptoolbox_copy.mdp.PolicyIteration(P, R, discount=discount)
pi.policy0=[1,1,1,1,1]
#vi.setVerbose()
pi.run()

policy_pi = pi.policy

print "Optimal policy (policy iteration) : \n" , policy_pi

policy_pg  = policy_gradient_algo( P, R , discount , path_len ,  n_paths, gamma=10 , eps=0.01)

print "Optimal policy (policy gradient) :\n" , mu_policy(policy_pg)

Optimal policy (policy iteration) : 
(0, 0, 0, 0, 0)
Optimal policy (policy gradient) :
[[  9.84749183e-01   1.52508172e-02]
 [  1.54634257e-03   9.98453657e-01]
 [  7.96454703e-02   9.20354530e-01]
 [  9.98535462e-01   1.46453781e-03]
 [  1.00000000e+00   7.57949729e-13]]


In [24]:
import mdptoolbox_copy.example as mdp_ex

n_states = 5
n_actions = 2
fire_prob = 0.1
discount=0.9
n_paths=100
path_len=100
path_len = 10
path_num = 10

P, R = mdp_ex.forest(S=n_states, p=fire_prob)

def generate_rollout(mu_policy, transition_matrix, reward_matrix, discount, path_len=10) : 
    n_states, n_actions = mu_policy.shape
    path_action = np.zeros(path_len , dtype=int)
    path_state = np.zeros(path_len, dtype=int)
    path_state[0] = rand.random_integers(n_states) -1
    path_action[0] = np.random.choice(n_actions, 1, p=mu_policy[path_state[0]])
    path_reward =  reward_matrix[path_state[0], path_action[0]]
    for i in range(1, path_len) :
        path_state[i] = np.random.choice(n_states , 1, p=transition_matrix[path_action[i-1]][path_state[i-1]])
        path_action[i] = np.random.choice(n_actions, 1, p=mu_policy[path_state[i]])
        path_reward =  path_reward + discount**i * reward_matrix[path_state[i], path_action[i]]
    return path_state, path_action, path_reward


def generate_rollouts(mu_policy, transition_matrix, reward_matrix, discount, path_len=10, n_paths=10):
    n_states, n_actions = mu_policy.shape 
    paths_states = np.zeros(( n_paths , path_len), dtype=int)
    paths_actions = np.zeros(( n_paths , path_len), dtype=int)
    paths_rewards = np.zeros( n_paths  )
    for i in range(0, n_paths) : 
        paths_states[i], paths_actions[i], paths_rewards[i] = generate_rollout(mu_policy, transition_matrix, reward_matrix, discount, path_len)
    return paths_states, paths_actions, paths_rewards

Now we need function to generate rollouts from our policy. This will generate trajectories of our model under policy

In [25]:
paths_states, paths_actions, paths_rewards = generate_rollouts(mu, P, R, discount, path_len=5, n_paths=10)
print paths_states, paths_actions, paths_rewards

[[1 0 0 1 0]
 [3 4 4 0 0]
 [3 0 0 1 2]
 [3 4 0 1 0]
 [1 2 0 0 1]
 [3 0 0 0 0]
 [4 0 1 0 1]
 [3 0 0 1 0]
 [2 3 4 4 4]
 [4 0 1 0 1]] [[0 1 0 0 1]
 [0 0 1 1 0]
 [1 1 0 0 0]
 [0 1 0 1 1]
 [0 1 1 0 1]
 [1 1 1 1 0]
 [0 0 1 0 0]
 [1 1 0 1 1]
 [0 0 0 0 1]
 [1 0 1 0 1]] [ 0.      5.22    1.      2.529   1.5561  1.      4.81    1.729   7.4682
  3.4661]
